## Magic Methods and the Python data model

We've already seen 3 magic methods: `__init__`, `__repr__`, and `__str__`

It turns out that we can get a lot of power by providing implementations of a few more magic methods to make our classes behave like other types (arithmetic types collection types, mainly).

(For a list of all the magic methods you can override, see https://docs.python.org/3/reference/datamodel.html)

Let's consider a Pythonic deck of cards:


In [ ]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class Deck:
    # ranks and suits are class attributes because they
    # should be shared by all decks
    ranks = '2 3 4 5 6 7 8 9 10 J Q K A'.split()
    suits = 'clubs diamonds hearts spades'.split()

    def __init__(self):
#         self._cards = []
#         for suit in self.suits:
#             for rank in self.ranks:
#                 self._cards.append(Card(rank, suit))
        self._cards = [
            Card(rank, suit) 
            for suit in self.suits
            for rank in self.ranks
        ]

In [ ]:
d = Deck()

In [ ]:
# We can create a deck of cards, but it turns out it's not iterable...

for card in d:
    print(card)

In [ ]:
# ...unless we refer to `_cards` directly
for card in d._cards:
    print(card, end=' ')

In [ ]:
# we also cannot find the length of the deck
print(len(d))

In [ ]:
# ...at least not without referring to `_cards` directly
print(len(d._cards))

## Making our deck iterable
* the Python data model allows us to accomplish quite a bit, just by implement the `__len__()` and `__getitem__()` methods

```python
lst[5]  # lst.__getitem__(5)
```

In [ ]:
# a deck of cards, round two
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class Deck:
    ranks = '2 3 4 5 6 7 8 9 10 J Q K A'.split()
    suits = 'clubs diamonds hearts spades'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]
    def __len__(self):
        return len(self._cards)   # return self._cards.__len__()

    def __getitem__(self, position):
        # implements self[position]
        return self._cards[position]
        # return self._cards.__getitem__(position)

In [ ]:
deck = Deck()
len(deck)

In [ ]:
for card in deck:
    print(card, end=' ')

### ...but just by implementing \_\_`getitem`\_\_`()`, we get so much more!

In [ ]:
# like indexing
deck[0], deck[-1]  # deck[len(deck) - 1]

In [ ]:
# ...and slicing!
deck[9:13]   # => deck._cards[9:13]

In [ ]:
deck[12::13]   # start: stop: stride/step

## What about a method to pick a random card?
* no need because Python already has a function to choose a random item from a sequence

In [ ]:
from random import choice

help(choice)

In [ ]:
choice(deck)

In [ ]:
# Jupyter magic
choice??

## Two big advantages of using special methods to leverage the Python data model
*  users of your classes don’t have to memorize arbitrary method names for standard operations (“How to get the number of items? Is it __`.size()`__, __`.length()`__, or what?”)
* it’s easier to benefit from the rich Python standard library and avoid reinventing the wheel, e.g., __`random.choice()`__

In [ ]:
class Deck():
    ranks = '2 3 4 5 6 7 8 9 10 J Q K A'.split()
    suits = 'clubs diamonds hearts spades'.split()

    def __init__(self):
        self._cards = [
            Card(rank, suit) 
            for suit in self.suits                                        
            for rank in self.ranks
        ]
        
    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        # implements self[position]
        return self._cards[position]
    
    def __setitem__(self, position, value):
        # implements self[position] = value  ==> self.__setitem__(position, value)
        self._cards[position] = value


In [ ]:
# Also del deck[5] => deck.__delitem__(5)

In [ ]:
import random
deck = Deck()

In [ ]:
random.shuffle(deck)

list(deck)

In [ ]:
# also jupyter magic
random.shuffle??

# Attribute access

The magic method `__getattr__` is called by Python *whenever there would otherwise be an `AttributeError` raised*:

In [ ]:
class A:
    def __getattr__(self, name):
        print(f'Calling __getattr__({name})')
        return None

In [ ]:
a = A()
a.foo = 'bar'

In [ ]:
a.foo  # `__getattr__` not called

In [ ]:
print(a.bat_anything_else)

In [ ]:
a.__dict__

In [ ]:
class Proxy:
    def __init__(self, real):
        self._real = real
        
    def __getattr__(self, name):
        print('__getattr__', name)
        if name.startswith('_'):
            raise AttributeError
        return getattr(self._real, name)

In [ ]:
lst = []
p = Proxy(lst)

In [ ]:
p.append('5')
# _tmp = getattr(p, 'append')
# _tmp('5')

In [ ]:
p.append

In [ ]:
lst

Magic methods do not get forwarded

In [ ]:
repr(p)  # p._real.__repr__ is not looked up

In [ ]:
repr(p._real)

# Lab

Open the [OOP Magic Lab][oop-magic-lab]

[oop-magic-lab]: ./oop-magic-lab.ipynb